Firstly we create a DTM as we created in our previous document

In [20]:
#importing all the required libraries
import textmining
import shorttext
import pandas as pd
import numpy as np
import numpy.random as random
import matplotlib.pyplot as plt
import math

Using the US Presidents speeches dataset to create a Document term matrix.

In [2]:
usprez = shorttext.data.inaugural()
docids=sorted(usprez.keys())
docsall=[]
for key in docids:
    s=''.join(usprez[key])
    docsall.append(s)

Next using the 'TermDocumentMatrix()' of the 'text mining' library to create the DTM.

In [4]:
tdm = textmining.TermDocumentMatrix()
for d in docsall:
    tdm.add_doc(d)
#tdm.write_csv('matrix.csv', cutoff=1)
c=0
ans=[]
for row in tdm.rows(cutoff=1):
    if(c==0):
        c=1
        continue
    ans.append(row)

Using numpy functions to find the dimensions and different features of the matrix.

In [5]:
a=np.array(ans)
print(a.shape)

(56, 8971)


# CUR Decomposition

First calculating the square of the Frobenius norm of DTM.

In [6]:
frobenius_norm_square=np.sum(a**2)
frobenius_norm_square

6665205

Finding the probability using length square sampling.

In [9]:
#probability list for finding C
probability_list_columns=[]
sum=0;
for i in range(np.size(a,1)):
    q=np.array(a[:,i])
    probability_list_columns.append((np.sum(q**2))/(1.0*frobenius_norm_square))

In [8]:
#probability list for finding R
probability_list_rows=[]
sum=0;
for i in range(np.size(a,0)):
    q=np.array(a[i,:])
    probability_list_rows.append((np.sum(q**2))/(1.0*frobenius_norm_square))

Find the Matrix C.

In [62]:
#let s=100
s=100
columns=np.size(a,1)
all_index=random.choice(columns,s,probability_list_columns)

In [63]:
t=all_index[0]
C=np.zeros((56,s))
C[:,0]=a[:,t]
const=1.0/(math.sqrt(s*probability_list_columns[t]))
C=np.array(C*const)
for i in range(1,s):
    t=all_index[i]
    new=a[:,t]
    const=1.0/(math.sqrt(s*probability_list_columns[t]))
    new=np.array(new*const)
    C[:,i]=new

Finding the Matrix R.

In [64]:
#let r=30
r=30
rows=np.size(a,0)
all_index2=random.choice(rows,r,probability_list_rows)

In [65]:
R=np.zeros((r,8971))
R2=np.zeros((r,8971))
for i in range(r):
    t=all_index2[i]
    new=a[t,:]
    R2[i,:]=np.array(new)
    const=1.0/(math.sqrt(r*probability_list_rows[t]))
    new=np.array(new*const)
    R[i,:]=new

Finding the Matrix U from Matrix C and R.

In [66]:
#Firstly creating a matrix Y
Y=np.zeros((r,s))
for i in range(s):
    t=all_index[i]
    Y[:,i]=R2[:,t]

In [75]:
#Finding the SVD of matrix Y
from scipy.linalg import svd
u,s1,vt=svd(Y)

In [79]:
Sigma = np.zeros((r,s))
Sigma[:s, :r]=np.diag(s1)
Sigma_piv=np.linalg.pinv(Sigma)

In [88]:
#Finally finding U
Sigma_piv_sq=Sigma_piv**2
U=np.matmul(vt.transpose(),Sigma_piv_sq)
U=np.matmul(U,u.transpose())

Finding the CUR Approximation.

In [90]:
a_new=np.matmul(C,U)
a_new=np.matmul(a_new,R)

Finding the Frobenius error.

In [91]:
x=np.subtract(a,a_new)
frobenius_error=np.sum(x**2)
print(frobenius_error)

103372319630.50641
